In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [2]:
tg_api_token = '5102696637:AAHcm9cOmIF7-GyG7nHHJYA_yZCGSWSTu60'
tg_chat_id = '1126129331'

import requests

def send_tg_message(text='Cell execution completed.'):
    requests.post(
        'https://api.telegram.org/' +
        'bot{}/sendMessage'.format(tg_api_token), 
        params=dict(chat_id=tg_chat_id, text=text)
    )

# Get Data

In [3]:
pacient_state = {
    0: "Not Defined/Transient",
    1: "Baseline",
    2: "Stress",
    3: "Amusement",
    4: "Meditation",
    5: "Error/Not Labeled"
}

In [4]:
chest_signal_available = ['ECG', 'EMG', 'EDA', 'Temp', 'Resp']

In [5]:
subjects_available = ['S5', 'S2', 'S3', 'S4', 'S17', 'S10', 'S11', 'S16', 'S8', 'S6', 'S7', 'S9', 'S13', 'S14', 'S15']

In [6]:
try:
    subjects_available.remove(".DS_Store")
except:
    print(subjects_available)

print(subjects_available)

['S5', 'S2', 'S3', 'S4', 'S17', 'S10', 'S11', 'S16', 'S8', 'S6', 'S7', 'S9', 'S13', 'S14', 'S15']
['S5', 'S2', 'S3', 'S4', 'S17', 'S10', 'S11', 'S16', 'S8', 'S6', 'S7', 'S9', 'S13', 'S14', 'S15']


In [7]:
f=open("all_data",'rb')
data=pickle.load(f,encoding='latin1')

In [8]:
# input_chest_data = get_input_chest_data(subjects_available)
input_chest_data = data["input_data"]

In [9]:
len(input_chest_data)

15

# Cleaning Data

In [10]:
# win_size = 60 # will represent 1 minute
# win_size = 5*700 # will represent 5 seconds
# step_slinding_window = 700//4 # moving the window in 0.25 sec
win_size = (90*700)//100
step_slinding_window = 100

# win_size = 630

def simple_rolling_z_score(x, win_size):
    r = x.rolling(window=win_size)
    m = r.mean()
    s = r.std(ddof=0)
    z = (x-m)/s
    return z

In [11]:
def rolling_z_score(df, win_size, step_slinding_window = 1):

    copy_df = df.copy()

    for i in range(len(df) - win_size + 1):
        slicer = slice(i, win_size + i, step_slinding_window)
        mean = df[slicer].mean()
        std = df[slicer].std(ddof=0)
        
        print(f"Normalazing Window {i}", end="\r", flush=True)

        copy_df[slicer] = (df[slicer] - mean) / std

    return copy_df


In [12]:
clean_chest_data = []
i = 0

print(f"Window Size: {win_size}, and Step: {step_slinding_window}\n\n")

for data_df in input_chest_data:

    # Resampling data choosing one point each 700 points
    resample_data_df = data_df[::100]

    # Separating features and targets
    X, y = resample_data_df[chest_signal_available], resample_data_df["label_id"]

    # Getting subject label
    subject_label = resample_data_df.pop("subject").to_numpy()[0]

    # Oversampling data
    # X, y = oversample.fit_resample(X, y)

    # Convert target to numpy
    target = y.to_numpy()
    
    send_tg_message(f"Normalizando o Subject {subject_label}. Ainda faltam {len(input_chest_data) - i}.")
    i = i + 1

    print(f"Start Normalization of Subject {subject_label}\n\n", end="\r", flush=True)

    # applying rolling score-z
    feature_df = rolling_z_score( X, win_size)

    print(f"\n\nEnd of Normalization of Subject {subject_label}", end="\r", flush=True)

    clean_chest_data.append((feature_df, target, subject_label))
    
send_tg_message("Terminou de Normalizar Geral")

del input_chest_data
gc.collect()

Window Size: 630, and Step: 100


Start Normalization of Subject S5

Normalazing Window 43176

Start Normalization of Subject S25

Normalazing Window 41923

Start Normalization of Subject S32

Normalazing Window 44821

Start Normalization of Subject S43



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Normalazing Window 38787

Start Normalization of Subject S816



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Normalazing Window 38206

Start Normalization of Subject S154



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
with open("cleaned_data", 'wb') as f:
    pickle.dump({
        "cleaned_data": clean_chest_data
    }, f)

In [14]:
print("FINISHED")
send_tg_message("Salvou os dados também")

FINISHED
